<a href="https://colab.research.google.com/github/amadeus-art/azure-openai-coding-dojo/blob/main/azure_openai_react.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reasoning + Act: make GPT interact with tools

## Install Depedencies

In [1]:
!pip install langchain
!pip install openai
!pip install python-dotenv

# for serpapi
!pip install google-search-results
# needed to "escape" SSL inside corporate network
!pip install python-certifi-win32

## Environment Setup
Before executing the following cells, make sure to set the `AZURE_OPENAI_KEY` and `AZURE_OPENAI_ENDPOINT` variables in the `.env` file or export them.

<br/>
<img src="assets/keys_endpoint.png" width="800"/>
<br/>

In addition, for this notebook we will use Google Search API, which requires an API Key that you can have for free registering with you google email here (100 searches/month): https://serpapi.com

Once you have it, set the `SERPAPI_API_KEY` variable in the `.env` file or export it.

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
api_key = os.getenv("AZURE_OPENAI_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
deployment = 'gpt-35-dojo'
openai.api_version = '2023-05-15' # may change in the future
openai.api_type = 'azure'

serpapi_api_key = os.getenv('SERPAPI_API_KEY')

In [4]:
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType
)

from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_key=api_key,
    openai_api_base=api_base,
    deployment_name=deployment,
    openai_api_version='2023-05-15',
    temperature=0
)

In [5]:
tools = load_tools([
    "serpapi",
    "llm-math",
    ],
    llm=llm,
    serpapi_api_key=serpapi_api_key
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [6]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [7]:
# run the agent
agent.run("How much is Amadeus IT Group net worth in 2022, raised to the power of 0.678? ")



> Entering new AgentExecutor chain...
I need to find the net worth of Amadeus IT Group and then raise it to the power of 0.678.
Action: Search
Action Input: "Amadeus IT Group net worth 2022"
Observation: Revenue in 2022 (TTM): $4.37 B According to Amadeus IT Group's latest financial reports the company's current revenue (TTM) is $4.37 B.
Thought:Now I need to calculate 4.37 raised to the power of 0.678.
Action: Calculator
Action Input: 4.37^0.678
Observation: Answer: 2.7179809861505504
Thought:I now know the final answer.
Final Answer: Amadeus IT Group's net worth in 2022 raised to the power of 0.678 is approximately $2.72 billion.

> Finished chain.


"Amadeus IT Group's net worth in 2022 raised to the power of 0.678 is approximately $2.72 billion."

## Add Memory to the Agent
In this section we showcase how to add a buffer memory containing the conversation history to the previous agent. This will allow us to ask follow-up questions having the previous computation into the context.

In [8]:
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""

# Notice how here we have {chat_history}!
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [9]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
)

In [10]:
print(agent_chain.agent.llm_chain.prompt.template)

Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}


In [11]:
agent_chain.run("How much is Amadeus IT Group net worth in 2022, raised to the power of 0.678? ")



> Entering new AgentExecutor chain...
Thought: I need to find the net worth of Amadeus IT Group in 2022 and raise it to the power of 0.678.
Action: Search
Action Input: "Amadeus IT Group net worth 2022"
Observation: Revenue in 2022 (TTM): $4.37 B According to Amadeus IT Group's latest financial reports the company's current revenue (TTM) is $4.37 B.
Thought:Now I need to calculate 4.37 raised to the power of 0.678.
Action: Calculator
Action Input: 4.37^0.678
Observation: Answer: 2.7179809861505504
Thought:I now know the final answer.
Final Answer: Amadeus IT Group's net worth in 2022 raised to the power of 0.678 is approximately $2.72 billion.

> Finished chain.


"Amadeus IT Group's net worth in 2022 raised to the power of 0.678 is approximately $2.72 billion."

In [12]:
agent_chain.run("What is the square root of the last number you computed ?")



> Entering new AgentExecutor chain...
Thought: I need to find the square root of $2.72 billion.
Action: Calculator
Action Input: sqrt(2.72*10^9)
Observation: Answer: 52153.61924162119
Thought:I now know the square root of $2.72 billion.
Final Answer: The square root of $2.72 billion is approximately 52153.62.

> Finished chain.


'The square root of $2.72 billion is approximately 52153.62.'

In [13]:
agent_chain.run("how many employees does Amadeus IT Group have ? How much is that number multiplied by the answer of the previous square root ?")



> Entering new AgentExecutor chain...
Thought: I need to find the number of employees at Amadeus IT Group and multiply it by the square root of their net worth.
Action: Search
Action Input: "Amadeus IT Group number of employees"
Observation: 16,433
Thought:Now I need to multiply 16,433 by 52153.62.
Action: Calculator
Action Input: 16433 * 52153.62
Observation: Answer: 857040437.46
Thought:I now know the final answer.
Final Answer: The final answer is $857,040,437.46.

> Finished chain.


'The final answer is $857,040,437.46.'